In [6]:
from xception import Xception, pretrained_settings
import torch
import torch.nn as nn
import torch.nn.functional as F

xception_path = 'checkpoints/xception-b5690688.pth'
def xception(num_classes=1000, pretrained='imagenet'):
    model = Xception(num_classes=num_classes)
    if pretrained:
        settings = pretrained_settings['xception'][pretrained]
        assert num_classes == settings['num_classes'], \
            "num_classes should be {}, but is {}".format(settings['num_classes'], num_classes)

        state_dict = torch.load(xception_path)
        for name, weights in state_dict.items():
            if 'pointwise' in name:
                state_dict[name] = weights.unsqueeze(-1).unsqueeze(-1)

        model.load_state_dict(state_dict)

        model.input_space = settings['input_space']
        model.input_size = settings['input_size']
        model.input_range = settings['input_range']
        model.mean = settings['mean']
        model.std = settings['std']

    # TODO: ugly
    model.last_linear = model.fc
    del model.fc
    return model


class TransferModel(nn.Module):
    """
    Simple transfer learning model that takes an imagenet pretrained model with
    a fc layer as base model and retrains a new fc layer for num_out_classes
    """
    def __init__(self, model, num_out_classes=2, dropout=0.0):
        super(TransferModel, self).__init__()
        self.model = model
        # Replace fc
        num_ftrs = self.model.last_linear.in_features
        if not dropout:
            self.model.last_linear = nn.Linear(num_ftrs, num_out_classes)
        else:
            print('Using dropout', dropout)
            self.model.last_linear = nn.Sequential(
                nn.Dropout(p=dropout),
                nn.Linear(num_ftrs, num_out_classes)
            )

         # Initialize the loss function
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, pixel_values, labels=None):
        logits = self.model(pixel_values)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        # Return loss and other model-specific outputs
        return (loss, logits) if loss is not None else logit


In [7]:
old_model = xception()
model = TransferModel(model=old_model,
                             num_out_classes=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [8]:
from torch.utils.data import Dataset
import os
from PIL import Image, ImageFile
import random
import pandas as pd

# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True
def shuffled_image_paths(folder_true, folder_false):
   # Gather true and false paths
    true_paths = [{'path': os.path.join(folder_true, f), 'label': 'Real'} for f in os.listdir(folder_true) if f.endswith(('jpg', 'png', 'jpeg'))]
    false_paths = [{'path': os.path.join(folder_false, f), 'label': 'Fake'} for f in os.listdir(folder_false) if f.endswith(('jpg', 'png', 'jpeg'))]

    all_paths = true_paths + false_paths
    random.shuffle(all_paths)  # Shuffle the combined list
    df = pd.DataFrame(all_paths)
    return df

from datasets import Dataset, Features, Value, ClassLabel

def create_path_label_dataset(folder_true, folder_false):
    paths_labels = shuffled_image_paths(folder_true, folder_false)

    # Define dataset features
    features = Features({
        'path': Value('string'),
        'label': ClassLabel(names=['Real', 'Fake']),
    })

    # Create the dataset
    dataset = Dataset.from_pandas(paths_labels, features=features)

    return dataset

from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=old_model.mean, std=old_model.std)
size = old_model.input_size[1:]
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

def transforms(examples):
    images = [Image.open(path).convert("RGB") for path in examples['path']]
    examples["pixel_values"] = [_transforms(img) for img in images]
    del examples["path"]
    return examples

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [9]:
folder_true = 'images'
folder_false = 'images/fake'

dataset = create_path_label_dataset(folder_true, folder_false)
dataset = dataset.train_test_split(test_size=0.2)
dataset = dataset.with_transform(transforms)

labels = dataset['train'].features["label"].names

FileNotFoundError: [Errno 2] No such file or directory: 'images'

In [1]:
import evaluate
import numpy as np
from transformers import TrainingArguments, Trainer

accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="deepfake-0",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics,
)

NameError: name 'model' is not defined

In [ ]:
trainer.train()

In [ ]:
model(transforms([Image.open(dataset['train'][0]['path'])]))